# Binary Sentiment Analysis on IMDB Reviews Dataset

# Libraries & Modules

In [1]:
# Manage Warnings
import warnings
warnings.filterwarnings("ignore")

# Modules
import os
import sys
import shutil
import string
import re as regex
import matplotlib.pyplot as plt

# TensorFlow Log Level
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# TensorFlow Libraries
import tensorflow as tf

# Environment/Versions
print("Python Env : ", sys.version)
print("TensorFlow : ", tf.__version__)

Python Env :  3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
TensorFlow :  2.12.0


# Dataset

## Downloading & Exploring

In [2]:
# Dataset Download Path
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

# Retrieve File (and untar)
dataset = tf.keras.utils.get_file(fname="aclImdb_v1", 
                                  origin=url, 
                                  untar=True, 
                                  cache_dir=".", 
                                  cache_subdir="")

# Dataset Directory
dataset_dir = os.path.join(os.path.dirname(dataset), "aclImdb")

84125825/84125825 [==============================] - 2s 0us/step


In [3]:
# List Dataset Sub-Directories & Files
display(os.listdir(dataset_dir))

['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

In [4]:
# Review Readme File
with open(os.path.join(dataset_dir, 'README'), 'r') as f:
    print(f.read())

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an additional 50,000 unlabeled
documents for unsupervised learning. 

In the entire collection, no more than 30 reviews are allowed for any
given movie because reviews for the same movie tend to have correlated
ratings. Further, the train and test sets contain a disjoint set of
movies, so no significant performance is obtained by memorizing
movie-unique terms and their associated with observed labels.  In the
labeled train/test sets, a negative review has a score <= 4 out of 10,
and a positive review has a scor

In [5]:
# Train & Test Directories
display(os.listdir(f"{dataset_dir}/train"))
display(os.listdir(f"{dataset_dir}/test"))

['urls_unsup.txt',
 'urls_neg.txt',
 'urls_pos.txt',
 'neg',
 'unsup',
 'unsupBow.feat',
 'pos',
 'labeledBow.feat']

['urls_neg.txt', 'urls_pos.txt', 'neg', 'pos', 'labeledBow.feat']